In [2]:
import pyodbc
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import scipy.stats as stats

In [3]:
# SQL Authentication
conn = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=vfm-sql-prod.verz.local\SNS, 1433;'
    'Database=vfm;'
    'UID=snsconnect;'
    'PWD=%Sungard765;')

In [ ]:
Hier lezen we de data in een pandas dataframe

In [4]:
sql_query = pd.read_sql_query(
'''
select * 
from [dbo].[snsv_rates_all_r]
where rate_dt >= '1-oct-2020'
               ''', conn)

df = pd.DataFrame(sql_query)


In [12]:
grouped = df.groupby(['rate_name','ccy','time_band'] )

In [20]:
mean = grouped['mid'].mean()
mean

rate_name          ccy  time_band
02-year US T-Note  USD  Dec 2020     110.407460
                        Sep 2020     110.437500
05-year US T-Note  USD  Dec 2020     125.652344
                        Mar 2021     125.987166
                        Sep 2020     125.910160
                                        ...    
XS2225428809       EUR  ACTUAL       103.688810
XS2233120554       EUR  ACTUAL       103.405857
XS2240278692       EUR  ACTUAL       101.395690
XS2251371022       EUR  ACTUAL       103.810500
XS2259193998       EUR  ACTUAL       100.357769
Name: mid, Length: 2630, dtype: float64

In [16]:
std =  grouped['mid'].std()

In [28]:
pd.Series((grouped['mid'] - mean) / std)

ValueError: operands could not be broadcast together with shapes (2630,2) (2630,) 

In [19]:
df['mean'] = grouped['mid'].apply(pd.expanding_mean)

AttributeError: module 'pandas' has no attribute 'expanding_mean'

In [5]:
yc = df.loc[ (df['rate_name']=="Swap 6M BB") & (df['ccy']=='EUR') ]


In [6]:
today = date.today()

print (today)


2020-11-27


In [7]:
index = pd.date_range(end = today, periods = 20, freq='1D')

In [11]:
selected = yc.loc[yc["rate_dt"].isin(index)]


method

In [9]:
mean = selected['mid'].mean()
mean

TypeError: 'method' object is not subscriptable

In [71]:
std = selected['mid'].std()
std

0.20464342861385554

In [72]:
selected.head()

,rate_dt,rate_name,ccy,rate_type,seq_no,curve_type,input_dt,time_band,days_off,actual_dt,...,start_term,maturity_term,start_term_dt,maturity_term_dt,start_term_date_days,maturity_term_date_days,exchange_name,bid,offer,mid
56049,2020-11-09,Swap 6M BB,EUR,End of Day,4677597,Yield,2020-11-09,15 YEARS,2.0,2035-11-12,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.009,-0.031,-0.011
56050,2020-11-09,Swap 6M BB,EUR,End of Day,4677598,Yield,2020-11-09,19 YEARS,2.0,2039-11-11,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.085,0.045,0.065
56051,2020-11-09,Swap 6M BB,EUR,End of Day,4677599,Yield,2020-11-09,23 YEARS,2.0,2043-11-11,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.102,0.062,0.082
56052,2020-11-09,Swap 6M BB,EUR,End of Day,4677600,Yield,2020-11-09,27 YEARS,2.0,2047-11-11,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.088,0.048,0.068
56053,2020-11-09,Swap 6M BB,EUR,End of Day,4677601,Yield,2020-11-09,35 YEARS,2.0,2055-11-11,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.045,0.005,0.025


In [78]:
selected['zscore'] =  pd.Series([selected['mid']-mean / std])

<ipython-input-78-83839ab42416>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected['zscore'] =  pd.Series([selected['mid']-mean / std])


In [77]:
selected

,rate_dt,rate_name,ccy,rate_type,seq_no,curve_type,input_dt,time_band,days_off,actual_dt,...,start_term,maturity_term,start_term_dt,maturity_term_dt,start_term_date_days,maturity_term_date_days,exchange_name,bid,offer,mid
56049,2020-11-09,Swap 6M BB,EUR,End of Day,4677597,Yield,2020-11-09,15 YEARS,2.0,2035-11-12,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.009,-0.031,-0.011
56050,2020-11-09,Swap 6M BB,EUR,End of Day,4677598,Yield,2020-11-09,19 YEARS,2.0,2039-11-11,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.085,0.045,0.065
56051,2020-11-09,Swap 6M BB,EUR,End of Day,4677599,Yield,2020-11-09,23 YEARS,2.0,2043-11-11,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.102,0.062,0.082
56052,2020-11-09,Swap 6M BB,EUR,End of Day,4677600,Yield,2020-11-09,27 YEARS,2.0,2047-11-11,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.088,0.048,0.068
56053,2020-11-09,Swap 6M BB,EUR,End of Day,4677601,Yield,2020-11-09,35 YEARS,2.0,2055-11-11,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.045,0.005,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82239,2020-11-26,Swap 6M BB,EUR,End of Day,4703823,Yield,2020-11-26,30 YEARS,2.0,2050-11-30,...,NaN,NaN,NaT,NaT,NaN,NaN,None,0.018,-0.022,-0.002
82240,2020-11-26,Swap 6M BB,EUR,End of Day,4703824,Yield,2020-11-26,35 YEARS,2.0,2055-11-30,...,NaN,NaN,NaT,NaT,NaN,NaN,None,-0.010,-0.050,-0.030
82241,2020-11-26,Swap 6M BB,EUR,End of Day,4703825,Yield,2020-11-26,40 YEARS,2.0,2060-11-30,...,NaN,NaN,NaT,NaT,NaN,NaN,None,-0.037,-0.077,-0.057
82242,2020-11-26,Swap 6M BB,EUR,End of Day,4703826,Yield,2020-11-26,50 YEARS,2.0,2070-12-01,...,NaN,NaN,NaT,NaT,NaN,NaN,None,-0.065,-0.105,-0.085


In [75]:
[selected['mid']-mean / std]

[56049    0.558724
 56050    0.634724
 56051    0.651724
 56052    0.637724
 56053    0.594724
            ...   
 82239    0.567724
 82240    0.539724
 82241    0.512724
 82242    0.484724
 82243    0.549724
 Name: mid, Length: 476, dtype: float64]

In [32]:
dt = df['rate_dt'].max

print (dt)

<bound method Series.max of 0        2020-01-01
1        2020-01-01
2        2020-01-01
3        2020-01-02
4        2020-01-02
            ...    
647150   2020-11-26
647151   2020-11-26
647152   2020-11-26
647153   2020-11-26
647154   2020-11-26
Name: rate_dt, Length: 647155, dtype: datetime64[ns]>
